In [1]:
import torch
import random
import torchvision
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import MultiStepLR
from sklearn.model_selection import train_test_split

from utils.util import get_image_list, dataset_list, PathDataset, PathDataset_v2, make_task
from avalanche.benchmarks.datasets import CelebA

from avalanche.benchmarks.utils import AvalancheTensorDataset
from avalanche.training.supervised import AGEM
from avalanche.logging import InteractiveLogger, WandBLogger
from avalanche.benchmarks.generators import nc_benchmark, paths_benchmark
from avalanche.benchmarks.utils import AvalancheDataset, AvalancheDatasetType
from avalanche.benchmarks.utils.dataset_definitions import IDatasetWithTargets
from avalanche.training.plugins import EvaluationPlugin
from avalanche.training.plugins.lr_scheduling import LRSchedulerPlugin
from avalanche.evaluation.metrics import ExperienceAccuracy, ExperienceLoss, ExperienceForgetting, ExperienceCPUUsage, ExperienceMaxGPU, ExperienceMaxRAM, ExperienceTime, EpochAccuracy
from avalanche.benchmarks.generators import (
    filelist_scenario, dataset_scenario, tensor_scenario, paths_scenario
)

/home/cal-05/anaconda3/envs/hspark/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
seed = 0
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
# torch.cuda.manual_seed_all(seed) # if use multi-GPU
cudnn.deterministic = True  # 연산 처리 속도 감소 -> 모델과 코드를 배포해야 하는 연구 후반 단계에 사용
cudnn.benchmark = False
# torch.set_printoptions(sci_mode=False)

In [3]:
transform = transforms.Compose([
    # transforms.Resize(224),
    transforms.ToTensor()
    ])

In [4]:
incremental = 1000
data_path = "../Data/CASIA-WebFace/" 

dataset = get_image_list(data_path)

image_dataset = []
label_dataset = []
incremental_list = list(range(incremental))

for idx, label in enumerate(dataset[1]):
    if label in incremental_list:
        image_dataset.append(dataset[0][idx])
        label_dataset.append(dataset[1][idx])

# x_train, x_test, y_train, y_test = train_test_split(dataset[0], dataset[1], test_size=0.2, shuffle=True, random_state=seed, stratify=dataset[1])
x_train, x_test, y_train, y_test = train_test_split(image_dataset, label_dataset, test_size=0.2, shuffle=True, random_state=seed, stratify=label_dataset)

train_set = dataset_list(x_train, y_train)
test_set = dataset_list(x_test, y_test)

print("train : ", len(y_train))
print("test : ", len(y_test))

train :  39374
test :  9844


In [5]:
transforms_group = dict(
       train=(
       transforms.Compose(
              [
              transforms.ToTensor(),
              ]
       ),
       None,
       ),
       eval=(
       transforms.Compose(
              [
              transforms.ToTensor(),
              ]
       ),
       None,
       )
)

# train_set = AvalancheDataset(trian_loader)
# test_set = AvalancheDataset(test_loader)

In [6]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# num_class = 10575
# train_task = make_task(incremental, y_train)
# test_task = make_task(incremental, y_test)

# train_set = PathDataset(train_set, transforms_group, "train", train_task)
# test_set = PathDataset(test_set, transforms_group, "eval", test_task)
train_set = PathDataset_v2(train_set, transforms_group, "train")
test_set = PathDataset_v2(test_set, transforms_group, "eval")

scenario = nc_benchmark(train_dataset=train_set,
                        test_dataset=test_set,
                        n_experiences=incremental,
                        task_labels=False,
                        seed=seed,
                        shuffle=True
                        )

model = torchvision.models.resnet18(pretrained=False, num_classes=incremental)
model.to(device)

optimizer = optim.SGD(model.parameters(), lr=1e-1)
criterion = torch.nn.CrossEntropyLoss()

In [7]:
interactive_logger = InteractiveLogger()
wandb_logger = WandBLogger(project_name="Face Recognition Test", run_name="AGEM-CASIA-M1000-E1-C500")
eval_plugin = EvaluationPlugin(
    EpochAccuracy(),
    ExperienceAccuracy(),
    ExperienceLoss(),
    ExperienceForgetting(),
    ExperienceTime(),
    loggers=[interactive_logger, wandb_logger])

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: hopo55. Use `wandb login --relogin` to force relogin


/home/cal-05/anaconda3/envs/hspark/lib/python3.8/site-packages/avalanche/training/plugins/evaluation.py:81: UserWarning: No benchmark provided to the evaluation plugin. Metrics may be computed on inconsistent portion of streams, use at your own risk.
  warnings.warn(


## A-GEM

In [8]:
train_batch = 10
eval_batch = 1
epoch = 1
memory_size = 1000

strategies = AGEM(model, optimizer, criterion, patterns_per_exp=memory_size, sample_size=memory_size, train_epochs=epoch, device=device, train_mb_size=train_batch, eval_mb_size=eval_batch, evaluator=eval_plugin)

In [9]:
print("Starting experiment...")
results = []
for experience in scenario.train_stream:
    print("Start training on experience ", experience.current_experience)
    strategies.train(experience)
    print("End training on experience ", experience.current_experience)
    print("Computing accuracy on the test set")
    results.append(strategies.eval(scenario.test_stream[:]))

Starting experiment...
Start training on experience  0
-- >> Start of training phase << --
100%|██████████| 3/3 [00:00<00:00, 10.64it/s]
Epoch 0 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.6296
-- >> End of training phase << --
End training on experience  0
Computing accuracy on the test set
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 7/7 [00:00<00:00, 52.22it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task000/Exp000 = 0.0236
	Time_Exp/eval_phase/test_stream/Task000/Exp000 = 0.1352
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 1.0000
-- Starting eval on experience 1 (Task 0) from test stream --
100%|██████████| 4/4 [00:00<00:00, 50.33it/s]
> Eval on experience 1 (Task 0) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task000/Exp001 = 10.0267
	Time_Exp/eval_phase/test_stream/Task000/Exp001 = 0.0796
	Top1_Acc_Exp/eval_phase/test_stream/Task

RuntimeError: CUDA out of memory. Tried to allocate 1.89 GiB (GPU 0; 10.76 GiB total capacity; 9.19 GiB already allocated; 120.44 MiB free; 9.24 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

: 